# Outdated

I won't use the weighted by citations approach anymore

In [1]:
import string
import pandas as pd
import pyLDAvis.gensim
import pyLDAvis.sklearn
import re
import pickle
from tqdm.notebook import tqdm
import unicodedata2
import numpy as np
import random
import os

#from libs.lda_wrapper import LDA_wrapper
from libs.LastNamesInference import LastNamesInference

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
tqdm.pandas()

/home/user/miniconda/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Load data

In [6]:
#df_full = pd.read_csv('/data/WOS/US/text_clean.txt')
df_socsci = pd.read_csv('/data/datasets/WOS/US/text_clean_socsci.txt')
us_papers = pd.read_csv('/data/datasets/WOS/US/US_papers.txt')

In [8]:
us_papers['cit_rel_all_IAC'] = us_papers['cit_rel_all_IAC'].fillna(0)

## utils

In [9]:
def save(x, file_name):
    with open(file_name, 'wb') as handle:
        pickle.dump(x, handle, protocol=pickle.HIGHEST_PROTOCOL)

def restore(file_name):
    with open(file_name, 'rb') as handle:
        x = pickle.load(handle)
    return x

In [10]:
def infer_race(us_papers,df_socsci, authors='first'):
    socsci_papers = us_papers.loc[(us_papers.id_art.isin(df_socsci.id_art)),]
    first_authors = socsci_papers[socsci_papers.ordre==1].copy().reset_index(drop=True)
    #first_authors = first_authors[['id_art','Prenom', 'nom']]
    lni = LastNamesInference(names = first_authors.nom)
    tqdm.pandas(desc="inferring race from lastnames")
    lastname_race_dist = first_authors.progress_apply(lambda x: lni.get_name_dist(lastname=x.nom), axis=1)
    first_authors[lni.prob_order] = pd.DataFrame(lastname_race_dist.to_list())
    #first_authors = first_authors[['id_art','white', 'hispanic', 'black', 'asian']]
    df_socsci_race = df_socsci.merge(first_authors, on ='id_art')

    return df_socsci_race

In [11]:
def restore_model(df,dataset= 'full',n_components=100):
    if not os.path.exists('../results/lda_model_{}_k{}.p'.format(dataset,n_components)):
        warnings.warn('we do not have that model')
    else:
        lda_model = restore( '../results/lda_model_{}_k{}.p'.format(dataset,n_components))
        vectorizer = restore( '../results/vectorizer_{}.p'.format(dataset))
    return lda_model,vectorizer

In [12]:
def transform_data(df,lda_model,vectorizer):
    texts = df.text_clean.values
    data_vectorized = vectorizer.transform(texts)
    doc_dist = lda_model.transform(data_vectorized)
    return data_vectorized, doc_dist

In [13]:
def weighted_race_dist(df_race):
    df_race = df_race.copy()
    df_race.loc[:,['white', 'hispanic', 'black', 'asian']] = df_race[['white', 'hispanic', 'black', 'asian']].multiply(df_race["cit_rel_all_IAC"], axis="index")
    return df_race
    

In [14]:
def project_lda_topics(df_race,doc_dist):
        
    race_dist = df_race.filter(regex=('white|hispanic|black|asian'))
    topics_by_group = race_dist.T @ doc_dist 
    
    joint_prob = topics_by_group/topics_by_group.to_numpy().sum()
    marginal_by_topic = joint_prob.div(joint_prob.sum(axis=0), axis=1)
    marginal_by_group = joint_prob.div(joint_prob.sum(axis=1), axis=0)
#    dist_diff_topic = marginal_by_topic.subtract(joint_prob.sum(axis=1), axis=0)  # with the substraction, this gives "how many percentual points (more/less) than 
                                                                                    # expected they talk about this topic
    dist_diff_topic = marginal_by_topic.div(joint_prob.sum(axis=1), axis=0) -1   # with the ratio, this gives "how much % (more/less) than expected
                                                                                    # they talk about this topic
    joint_prob = joint_prob.T.rename_axis('topic').reset_index()
    marginal_by_topic = marginal_by_topic.T.rename_axis('topic').reset_index()
    marginal_by_group = marginal_by_group.T.rename_axis('topic').reset_index()
    dist_diff_topic = dist_diff_topic.T.rename_axis('topic').reset_index()
    
    # I start the topics in 1, so they are equal to the LDAVIZ!!!!
    joint_prob.topic += 1
    marginal_by_topic.topic += 1
    marginal_by_group.topic += 1
    dist_diff_topic.topic += 1    
    
    return joint_prob, marginal_by_topic, marginal_by_group, dist_diff_topic

In [15]:
def intersect_by_gender(df_race, doc_dist, weighted = True):
    
    
    if weighted is True:
        df_race = weighted_race_dist(df_race)
    #race_dist = race_dist.merge(metadata_papers, how='left', left_index=True, right_on='id_art')
    #race_dist = first_or_so_authors(race_dist)

    df_race['gender'] = df_race.gender.str.upper()

    boolean_mask_M = df_race.gender == 'M'
    boolean_mask_F = df_race.gender == 'F'
    
    race_dist_M = df_race.loc[boolean_mask_M,['white','hispanic','black','asian']]
    race_dist_F = df_race.loc[boolean_mask_F,['white','hispanic','black','asian']]

    race_dist_M.columns = race_dist_M.columns + '_M'
    race_dist_F.columns = race_dist_F.columns + '_F'

    race_dist_MF = pd.concat([race_dist_M,race_dist_F]).fillna(0)
    doc_dist_MF = np.concatenate((doc_dist[boolean_mask_M],doc_dist[boolean_mask_F]))

    joint_prob, marginal_by_topic, marginal_by_group, dist_diff_topic = project_lda_topics(race_dist_MF,doc_dist_MF)

    return joint_prob, marginal_by_topic, marginal_by_group, dist_diff_topic

## race by paper

In [16]:
df_socsci_race = infer_race(us_papers,df_socsci)

imputing by the mean: 100%|██████████| 238652/238652 [00:01<00:00, 221769.21it/s]


In [17]:
lda_model,vectorizer = restore_model(df_socsci,dataset= 'socsci',n_components=300)

/home/user/miniconda/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator LatentDirichletAllocation from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/user/miniconda/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [18]:
lda_model

LatentDirichletAllocation(learning_method='online', n_components=300, n_jobs=-1,
                          random_state=1234, verbose=1)

In [19]:
data_vectorized_socsci_300, doc_dist_socsci_300 = transform_data(df_socsci_race,lda_model,vectorizer)

In [20]:
joint_prob_gender, marginal_by_topic_gender, marginal_by_group_gender, dist_diff_topic_gender = intersect_by_gender(df_socsci_race, doc_dist_socsci_300,weighted=True)

In [21]:
joint_prob_gender.to_csv('../results/joint_prob_gender_300_weighted.csv',index=False)
marginal_by_topic_gender.to_csv('../results/marginal_by_topic_gender_300_weighted.csv',index=False)
marginal_by_group_gender.to_csv('../results/marginal_by_group_gender_300_weighted.csv',index=False)
dist_diff_topic_gender.to_csv('../results/dist_diff_topic_gender_300_weighted.csv',index=False)


Topic proportion, weighted

In [22]:
doc_dist_socsci_300_weighted = (doc_dist_socsci_300.T * df_socsci_race.cit_rel_all_IAC.values).T

In [23]:
topic_proportion_weighted = pd.DataFrame(doc_dist_socsci_300_weighted.sum(axis=0)/np.sum(doc_dist_socsci_300_weighted.sum(axis=0)))

In [24]:
topic_proportion_weighted.columns = ['proportion']

In [25]:
topic_proportion_weighted['topic'] = topic_proportion_weighted.index +1

In [26]:
topic_proportion_weighted.to_csv('../results/topic_proportion_300_weighted.csv',index=False)

## Health

In [28]:
df_health = pd.read_csv('/data/datasets/WOS/US/text_clean_health.txt')


In [29]:
df_health_race = infer_race(us_papers,df_health)

imputing by the mean: 100%|██████████| 123472/123472 [00:00<00:00, 204004.67it/s]


In [30]:
lda_model_health,vectorizer_health = restore_model(df_health,dataset= 'health',n_components=200)


/home/user/miniconda/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator LatentDirichletAllocation from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/user/miniconda/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [ ]:
data_vectorized_health_200, doc_dist_health_200 = transform_data(df_health_race,lda_model_health,vectorizer_health)


In [ ]:
joint_prob_gender_health, marginal_by_topic_gender_health, marginal_by_group_gender_health, dist_diff_topic_gender_health = intersect_by_gender(df_health_race, doc_dist_health_200,weighted=True)

In [ ]:
joint_prob_gender_health.to_csv('../results/joint_prob_gender_health_200_weighted.csv',index=False)
marginal_by_topic_gender_health.to_csv('../results/marginal_by_topic_gender_health_200_weighted.csv',index=False)
marginal_by_group_gender_health.to_csv('../results/marginal_by_group_gender_health_200_weighted.csv',index=False)
dist_diff_topic_gender_health.to_csv('../results/dist_diff_topic_gender_health_200_weighted.csv',index=False)


Topic proportion, weighted

In [ ]:
doc_dist_health_200_weighted = (doc_dist_health_200.T * df_health_race.cit_rel_all_IAC.values).T

In [ ]:
topic_proportion_health_weighted = pd.DataFrame(doc_dist_health_200_weighted.sum(axis=0)/np.sum(doc_dist_health_200_weighted.sum(axis=0)))

In [ ]:
topic_proportion_health_weighted.columns = ['proportion']

In [ ]:
topic_proportion_health_weighted['topic'] = topic_proportion_health_weighted.index +1

In [ ]:
topic_proportion_health_weighted.to_csv('../results/topic_proportion_health_200_weighted.csv',index=False)

# Direct projection of citations & race

## Social Sciences

In [ ]:
def split_by_gender(df_race):
    df_race['gender'] = df_race.gender.str.upper()
    
    boolean_mask_M = df_race.gender == 'M'
    boolean_mask_F = df_race.gender == 'F'
       
    race_dist_M = df_race.loc[boolean_mask_M,['white','hispanic','black','asian']].copy()
    race_dist_F = df_race.loc[boolean_mask_F,['white','hispanic','black','asian']].copy()

    race_dist_M.columns = race_dist_M.columns + '_M'
    race_dist_F.columns = race_dist_F.columns + '_F'

    race_dist_MF = pd.concat([race_dist_M,race_dist_F]).fillna(0)
    
    assigned_gender_bool = np.asarray(boolean_mask_M) | np.asarray(boolean_mask_F)
    cit_rel_all_IAC_MF = df_race.loc[assigned_gender_bool,'cit_rel_all_IAC']
    return(race_dist_MF, cit_rel_all_IAC_MF, assigned_gender_bool)

In [ ]:
def average_citations_by_race(df_race):
    
    race_dist_MF, cit_rel_all_IAC_MF, _ = split_by_gender(df_race)
    
    n_citations = race_dist_MF.multiply(cit_rel_all_IAC_MF, axis="index").sum(axis=0)
    n_papers = race_dist_MF.sum(axis=0)

    avg_citations_race = n_citations/n_papers
    
    return avg_citations_race

In [ ]:
def average_citations_by_topic(df_race,doc_dist, split=False):
    
    if split is True:
        race_dist, cit_rel_all_IAC,assigned_gender_bool = split_by_gender(df_race)
        doc_dist = doc_dist[assigned_gender_bool]
    else:
        race_dist = df_race.filter(regex=('white|hispanic|black|asian')).copy()
        cit_rel_all_IAC = df_race["cit_rel_all_IAC"]

    citations_by_topics_and_group = race_dist.multiply(cit_rel_all_IAC, axis="index").T @ doc_dist
    papers_by_topics_and_group = race_dist.T @ doc_dist

    avg_citation_by_topic_race = citations_by_topics_and_group/papers_by_topics_and_group    
            
    return avg_citation_by_topic_race

In [ ]:
avg_citations_race = average_citations_by_race(df_socsci_race)

avg_citations_race.to_csv('../results/avg_citations_race_socsci.csv')

In [ ]:
average_citations_by_topic_gender_df = average_citations_by_topic(df_socsci_race,doc_dist_socsci_300,split=True)

average_citations_by_topic_gender_df = average_citations_by_topic_gender_df.T
average_citations_by_topic_gender_df['topic'] = average_citations_by_topic_gender_df.index + 1

average_citations_by_topic_gender_df.to_csv('../results/average_citations_by_topic_gender_socsci.csv', index=False)

In [ ]:
average_citations_by_topic_gender_df.loc[259]

## Health

In [ ]:
avg_citations_race = average_citations_by_race(df_health_race)

avg_citations_race.to_csv('../results/avg_citations_race_health.csv')

In [ ]:
average_citations_by_topic_gender_df = average_citations_by_topic(df_health_race,doc_dist_health_200,split=True)

average_citations_by_topic_gender_df = average_citations_by_topic_gender_df.T
average_citations_by_topic_gender_df['topic'] = average_citations_by_topic_gender_df.index + 1

average_citations_by_topic_gender_df.to_csv('../results/average_citations_by_topic_gender_health.csv', index=False)